### Limit the size of the original RID songs to get a reduced dataset with fewer padding values

In [2]:
# start by feeding your Pickle functions to call and save pickle variables later on

import pickle
def writePickle( Variable, fname):
    filename = fname +".pkl"
    f = open("pickle_vars/"+filename, 'wb')
    pickle.dump(Variable, f)
    f.close()
def readPickle(fname):
    filename = "pickle_vars/"+fname +".pkl"
    f = open(filename, 'rb')
    obj = pickle.load(f)
    f.close()
    return obj
def readPicklefromPast(fname):
    filename = "../pickle_vars/"+fname +".pkl"
    f = open(filename, 'rb')
    obj = pickle.load(f)
    f.close()
    return obj

In [ ]:
# load the previously formed RID datasets, prepared for training
train_RID_input_data = readPickle("cnn_data_inputs/train_RID_input_data")
train_RID_labels = readPickle("cnn_data_inputs/train_RID_labels")
dev_RID_input_data = readPickle("cnn_data_inputs/dev_RID_input_data")
dev_RID_labels = readPickle("cnn_data_inputs/dev_RID_labels")
test_RID_input_data = readPickle("cnn_data_inputs/test_RID_input_data")
test_RID_labels = readPickle("cnn_data_inputs/test_RID_labels")

In [ ]:
# an example
train_RID_input_data[0]

In [ ]:
import random

def RID_refiner(input_data):
    new_input_data = list()
    for song in input_data:
        reduced_song = []
        song_line_ruler = list()
        for line in song:
            if len(list(filter(lambda a: a != 7 and a != 0, line))) != 0:
                new_line = list(filter(lambda a: a != 7 and a != 0, line))
                song_line_ruler.append((song.index(line),len(new_line)))
            else:
                song_line_ruler.append((song.index(line),0))
        song_line_ruler = list(set(song_line_ruler)) # remove duplicate lines
        song_line_ruler.sort(key=lambda x: x[1], reverse=True) # sort by the number of RID tags present
        song_line_ruler = song_line_ruler[0:10] # take the most frequent 10
        song_line_ruler.sort(key=lambda x: x[0]) # sort them back in shape wrt their original places in the song

        for item in song_line_ruler:
            reduced_song.append(song[item[0]])
        refined_song = list()
        for line in reduced_song:
            if set(line) <= {0,7}: # if the line is all blanks, replace the line with three random integers up to 44
                refined_song.append([random.randint(0,44),random.randint(0,44),random.randint(0,44)])
            else:
                refined_line = list(filter(lambda a: a != 7 and a != 0, line))
                if len(refined_line) > 3: # if there are more than three elements, take the last 3 of them
                    refined_line = refined_line[-3:]
                else: # if there are only 1 or 2 non-0 and non-7 elements
                    diff = 3 - len(refined_line)
                    for _ in range(diff):
                        refined_line.insert(random.randrange(0,len(refined_line),1),random.randint(0,44))
                refined_song.append(refined_line)
        if len(refined_song) < 10:
            diff = 10 - len(refined_song)
            for _ in range(diff):
                refined_song.append([random.randint(0,44),random.randint(0,44),random.randint(0,44)])
            
        new_input_data.append(refined_song)
    return new_input_data

In [ ]:
train_RID_reduced_input_data = RID_refiner(train_RID_input_data)
dev_RID_reduced_input_data = RID_refiner(dev_RID_input_data)
test_RID_reduced_input_data = RID_refiner(test_RID_input_data)


In [ ]:
# convert all of them to numpy arrays, so that they can be used in keras
import numpy as np

train_RID_input_data = np.array(train_RID_reduced_input_data)
train_RID_labels = np.array(train_RID_labels)
dev_RID_input_data = np.array(dev_RID_reduced_input_data)
dev_RID_labels = np.array(dev_RID_labels)
test_RID_input_data = np.array(test_RID_reduced_input_data)
test_RID_labels = np.array(test_RID_labels)

In [ ]:
# for all the input data, we have to normalize the data points to an interval between 0 and 1, 
# and convert everything to floating numbers
train_RID_input_data = train_RID_input_data.astype('float32') / np.amax(train_RID_input_data)
dev_RID_input_data = dev_RID_input_data.astype('float32') / np.amax(dev_RID_input_data)
test_RID_input_data = test_RID_input_data.astype('float32') / np.amax(test_RID_input_data)

In [ ]:
# an example
train_RID_input_data[0]

In [ ]:
# reshape input

X_train_RID = train_RID_input_data.reshape(len(train_RID_input_data),10,3,1)
X_dev_RID = dev_RID_input_data.reshape(len(dev_RID_input_data),10,3,1)
X_test_RID = test_RID_input_data.reshape(len(test_RID_input_data),10,3,1)

# check shape
print(X_train_RID.shape)

In [ ]:
# then we need to turn the labels into categorical values

from keras.utils import to_categorical

y_train_RID = to_categorical(train_RID_labels)
y_dev_RID = to_categorical(dev_RID_labels)
y_test_RID = to_categorical(test_RID_labels)

In [ ]:
# record all of them to pickle files
writePickle(X_train_RID,"cnn_data_inputs/RID_Keras/X_train_RID_reduced")
writePickle(X_dev_RID,"cnn_data_inputs/RID_Keras/X_dev_RID_reduced")
writePickle(X_test_RID,"cnn_data_inputs/RID_Keras/X_test_RID_reduced")
writePickle(y_train_RID,"cnn_data_inputs/RID_Keras/y_train_RID_reduced")
writePickle(y_dev_RID,"cnn_data_inputs/RID_Keras/y_dev_RID_reduced")
writePickle(y_test_RID,"cnn_data_inputs/RID_Keras/y_test_RID_reduced")

### From the this moment on, we will use the Model scripts to construct our models, using the pickle input variables right above